In [2]:
import pandas as pd
from nsepython import *
import numpy as np
from nsepy import *
from datetime import datetime, timedelta

In [3]:
# Fetch data for Nifty 50
nifty_data = nsefetch('https://www.nseindia.com/api/equity-stockIndices?index=SECURITIES%20IN%20F%26O')

In [4]:
key_list = list(nifty_data.keys())

In [ ]:
key_list

In [6]:
df = pd.DataFrame(nifty_data['data'])
df = df.drop(0)

In [ ]:
len(df)

In [ ]:
df.columns

In [ ]:
df_test = df[['symbol','open', 'dayHigh', 'dayLow','yearHigh', 'yearLow','lastPrice']]
df_test.head()

In [ ]:
details_list = []
for i in range(1,182):
  x = df['meta'][i]
  del x['activeSeries']
  del x['debtSeries']
  del x['tempSuspendedSeries']
  del x['isFNOSec']
  del x['isCASec']
  del x['isSLBSec']
  del x['isDebtSec']
  del x['isSuspended']
  del x['isDelisted']
  del x['isETFSec']
  del x['quotepreopenstatus']
  del x['listingDate']
  #del x['slb_isin']
  del x['isMunicipalBond']

  details_list.append(pd.DataFrame([x]))

details_list[24]

In [ ]:
len(details_list)

In [12]:
df_details = pd.concat(details_list[:], ignore_index=True)

In [ ]:
df_details

In [14]:
df_details = df_details.drop(columns=['slb_isin'])

In [ ]:
df_details[df_details.isnull().any(axis=1)]

In [16]:
nse_df = pd.concat([df_test.set_index('symbol'), df_details.set_index('symbol')], axis = 1, join='outer').reset_index()

In [ ]:
nse_df.columns

In [18]:
nse_df = nse_df[['symbol','companyName', 'open', 'dayHigh', 'dayLow', 'yearHigh', 'yearLow', 'lastPrice', 'industry', 'isin']]

In [19]:
nse_df.rename(columns={'yearHigh':'52w_High', 'yearLow':'52w_Low', 'lastPrice':'Current_price'}, inplace = True)

In [ ]:
nse_df['symbol'][0]

In [21]:
keep_values = nse_df['symbol']
keep_values = keep_values.tolist()

In [ ]:
import pytz
time_zone = pytz.timezone('Asia/Kolkata')
current_time = datetime.now().astimezone(time_zone)
current_time = current_time.strftime('%H:%M')
current_time = current_time.split(':')
current_time = [int(x) for x in current_time]
print(current_time)

In [ ]:
current_time[1]

In [ ]:
current_date = datetime.now().date()
day_of_week = current_date.weekday()
if day_of_week == 5:
   current_date = current_date - timedelta(days = 1)
   print('date updated because it is saturday')

if (current_time[0] < 18):
    current_date = current_date - timedelta(days = 1)
    print('date updated because time is less than 6 pm')
    day_of_week = current_date.weekday()
    if day_of_week == 5:
     current_date = current_date - timedelta(days = 1)
     print('date updated because it is saturday')



current_date = current_date.strftime("%d-%m-%Y")
print(current_date)

In [ ]:
y = get_bhavcopy(current_date)
y = y[y['SYMBOL'].isin(keep_values)]
y.head()

In [ ]:
series_list = [' EQ']
y = y[y[' SERIES'].isin(series_list)]
len(y)

In [ ]:
y = y[['SYMBOL', ' DELIV_QTY']]
y.head()

In [28]:
nse_df = pd.concat([nse_df.set_index('symbol'), y.set_index('SYMBOL')], axis = 1, join='outer').reset_index(names = 'symbol')

In [ ]:
losing_df = get_bhavcopy(current_date)
losing_df = losing_df[losing_df['SYMBOL'].isin(keep_values)]
losing_df = losing_df[losing_df[' SERIES'].isin([' EQ'])]
account_losing_days = losing_df[['SYMBOL']]

account_losing_days['LOSING_DAYS'] = 0
print(len(account_losing_days))
account_losing_days.head()

In [30]:
# Get the current date
current_date = datetime.now().date()

# Define the number of days ago
days_ago = 34

# Calculate the date 20 days ago
date = current_date - timedelta(days=days_ago)

final_date = current_date

while(date != final_date):
  day_of_week = date.weekday()
  if day_of_week == 5:
    date = date + timedelta(days = 1)
    continue
  t = date
  t = t.strftime("%d-%m-%Y")


  # Wrap the get_bhavcopy call in a try-except block to handle HTTPError
  try:
    losing_df = get_bhavcopy(t)
  except urllib.error.HTTPError as e:
    print(f"Error fetching data for {t}: as NSE was closed on {date-timedelta(1)}")  # Print error message
    date = date + timedelta(days=1)  # Move to the next date
    continue  # Skip to the next iteration of the loop

  losing_df = losing_df[losing_df['SYMBOL'].isin(keep_values)]
  losing_df = losing_df[losing_df[' SERIES'].isin([' EQ'])]
  for index, row in losing_df.iterrows():
    if row[' OPEN_PRICE'] > row[' CLOSE_PRICE']:
        x = row['SYMBOL']
        index = account_losing_days.index[account_losing_days['SYMBOL'] == x].tolist()[0]
        account_losing_days.at[index, 'LOSING_DAYS'] += 1


  date = date + timedelta(days = 1)

In [ ]:
account_losing_days.head()

In [32]:
nse_df = pd.concat([nse_df.set_index('symbol'), account_losing_days.set_index('SYMBOL')], axis = 1, join='outer').reset_index(names = 'symbol')

In [33]:
nse_df['52w_High'] = nse_df['52w_High'].astype(float)
nse_df['52w_Low'] = nse_df['52w_Low'].astype(float)
nse_df['Current_price'] = nse_df['Current_price'].astype(float)

In [ ]:
nse_df['going_towards'] = np.nan
for index, row in nse_df.iterrows():
  high_diff = row['52w_High'] - row['Current_price']
  low_diff = row['Current_price'] - row['52w_Low']

  if (high_diff > low_diff):
    nse_df.at[index, 'going_towards'] = "Low"
  else:
    nse_df.at[index, 'going_towards'] = "High"

In [35]:
nse_df['open'] = nse_df['open'].astype(float)
nse_df['Current_price'] = nse_df['Current_price'].astype(float)
nse_df['dayHigh'] = nse_df['dayHigh'].astype(float)
nse_df['dayLow'] = nse_df['dayLow'].astype(float)
nse_df['52w_High'] = nse_df['52w_High'].astype(float)
nse_df['52w_Low'] = nse_df['52w_Low'].astype(float)

In [ ]:
nse_df['change%'] = np.nan
for index, row in nse_df.iterrows():
  change_pc = ((row['Current_price']-row['open'])/row['open'])*100
  change_pc = format(change_pc, ".2f")
  nse_df.at[index, 'change%'] = change_pc

In [37]:
import yfinance as yf

def profit_sales_analysis(ticker, parameter):
    cases = ['000', '001', '010', '011', '100', '101', '110', '111']

    try:
        ticker = f"{ticker}.NS"
        ticker_obj = yf.Ticker(ticker)
        financials = ticker_obj.financials

        y_4 = financials.iloc[financials.index.get_loc(parameter), 0]
        y_3 = financials.iloc[financials.index.get_loc(parameter), 1]
        y_2 = financials.iloc[financials.index.get_loc(parameter), 2]
        y_1 = financials.iloc[financials.index.get_loc(parameter), 3]

        result = ""

        if (y_4 - y_3) > 0:
            result += "1"
        else:
            result += "0"
        
        if (y_3 - y_2) > 0:
            result += "1"
        else:
            result += "0"
            
        if (y_2 - y_1) > 0:
            result += "1"
        else:
            result += "0"
        
        if result == cases[0]:
            return "Not Recommended"
        elif result == cases[1]:
            return "Check quarterly results"
        elif result == cases[2]:
            return "Caution need monitoring"
        elif result == cases[3]:
            return "Recommended"
        elif result == cases[4]:
            return "Not Recommended"
        elif result == cases[5]:
            return "Caution need Monitoring"
        elif result == cases[6]:
            return "Caution need Monitoring"
        elif result == cases[7]:
            return "Highly Recommended"


    except Exception as e:
        #print(f"Error fetching profit/sales data for {ticker}: {e}")
        return np.nan  # Return NaN if any error occurs

In [38]:
import yfinance as yf

def debt_analysis(ticker, parameter):
    cases = ['000', '001', '010', '011', '100', '101', '110', '111']
    
    try:
        ticker = f"{ticker}.NS"
        ticker_obj = yf.Ticker(ticker)
        balance_sheet = ticker_obj.balance_sheet

        y_4 = balance_sheet.iloc[balance_sheet.index.get_loc(parameter), 0]
        y_3 = balance_sheet.iloc[balance_sheet.index.get_loc(parameter), 1]
        y_2 = balance_sheet.iloc[balance_sheet.index.get_loc(parameter), 2]
        y_1 = balance_sheet.iloc[balance_sheet.index.get_loc(parameter), 3]

        result = ""

        if (y_4 - y_3) > 0:
            result += "1"
        else:
            result += "0"
        
        if (y_3 - y_2) > 0:
            result += "1"
        else:
            result += "0"
            
        if (y_2 - y_1) > 0:
            result += "1"
        else:
            result += "0"
        
        if result == cases[0]:
            return "Highly Recommended"
        elif result == cases[1]:
            return "Caution need monitoring"
        elif result == cases[2]:
            return "Caution need monitoring"
        elif result == cases[3]:
            return "Not Recommended"
        elif result == cases[4]:
            return "Recommended"
        elif result == cases[5]:
            return "Caution need monitoring"
        elif result == cases[6]:
            return "Caution need Monitoring"
        elif result == cases[7]:
            return "Not Recommended"


    except Exception as e:
        #print(f"Error fetching Debt data for {ticker}: {e}")
        return np.nan  # Return NaN if any error occurs

In [ ]:
x = np.nan
print(np.isnan(x))

In [ ]:
symbol_list = (nse_df['symbol']).tolist()
symbol_list

In [ ]:
symbol_list = (nse_df['symbol']).tolist()

Profit_Analysis = []
Sales_Analysis = []
Debt_Analysis = []

for symbol in symbol_list:

    profit = profit_sales_analysis(symbol, "EBIT")
    
    if not (isinstance(profit, str)):
        profit = profit_sales_analysis(symbol, "Net Income")
        print(f"Profit Value Changed for: {symbol}")
    
    sales = profit_sales_analysis(symbol, "Total Revenue")
    
    debt = debt_analysis(symbol, "Total Debt")
    
    Profit_Analysis.append(profit)
    Sales_Analysis.append(sales)
    Debt_Analysis.append(debt)

nse_df["Profit Analysis"] = Profit_Analysis
nse_df["Sales Analysis"] = Sales_Analysis
nse_df["Debt Analysis"] = Debt_Analysis
nse_df.head()

In [42]:
def get_company_details(ticker, detail):
    t = f"{ticker}.NS"
    t = yf.Ticker(t)
    return t.info[detail]

In [43]:
for index in range(181, len(nse_df)):
    if pd.isna(nse_df.at[index, "companyName"]):
        symbol = nse_df.at[index, "symbol"]
                
        # Try to get the company name (longName, fallback to shortName)
        try:
            name = get_company_details(symbol, "longName")
        except Exception as e:
            print(f"Error getting longName for symbol {symbol}: {e}")
            try:
                name = get_company_details(symbol, "shortName")
            except Exception as e:
                print(f"Error getting shortName for symbol {symbol}: {e}")
                name = None  # Use None or a default value if both fail
        
        # Get industry information
        try:
            industry = get_company_details(symbol, "industry")
        except Exception as e:
            print(f"Error getting industry for symbol {symbol}: {e}")
            industry = None  # Use None or a default value if this fails
        
        # Update the DataFrame
        nse_df.at[index, "companyName"] = name
        nse_df.at[index, "industry"] = industry


In [ ]:
nse_df['Profit Analysis'].isna().sum()

### News Scrapping and analysis


#### Performed in Colab

In [45]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.options import Options
import time

def scrape_yahoo_finance_news(symbol: str):
    print(symbol)
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")  # Added to help prevent crashes
    chrome_options.add_argument("--remote-debugging-port=9222")  # Added to help with DevTools issues
    chrome_options.add_argument("--disable-extensions")  # Disable extensions
    chrome_options.add_argument("--disable-software-rasterizer")  # Reduce GPU usage
    
    max_retries = 3
    for attempt in range(max_retries):
        try:
            driver = webdriver.Chrome(options=chrome_options)
            url = f"https://finance.yahoo.com/quote/{symbol}.NS/news/"
            
            try:
                driver.get(url)
                wait = WebDriverWait(driver, 3)  # Increased timeout
                
                try:
                    news_container = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-testid="news-tabs-container"]')))
                except TimeoutException:
                    return f"{symbol} - No news"
                
                try:
                    news_list = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.stream-items.yf-1usaaz9')))
                    news_items = news_list.find_elements(By.CSS_SELECTOR, 'li.stream-item.story-item.yf-1usaaz9')
                    
                    titles = []
                    descriptions = []
                    
                    for item in news_items:
                        try:
                            title_tag = item.find_element(By.CSS_SELECTOR, 'a.subtle-link.fin-size-small.titles.noUnderline.yf-1xqzjha h3')
                            description_tag = item.find_element(By.CSS_SELECTOR, 'a.subtle-link.fin-size-small.titles.noUnderline.yf-1xqzjha p.clamp.yf-18q3fnf')
                            titles.append(title_tag.text)
                            descriptions.append(description_tag.text)
                        except NoSuchElementException:
                            continue
                    
                    return titles, descriptions if titles and descriptions else f"{symbol} - No news"
                
                except TimeoutException:
                    return f"{symbol} - No news"
                
            finally:
                driver.quit()
                
        except Exception as e:
            print(f"Attempt {attempt + 1} failed for {symbol}: {str(e)}")
            if attempt == max_retries - 1:
                return f"{symbol} - Error: {str(e)}"
            time.sleep(2)  # Wait before retrying


In [46]:
from textblob import TextBlob

def news_analysis(news):
    
    # Check the input type
    if isinstance(news, str):
        return "NEUTRAL"

    if not isinstance(news, tuple) or len(news) != 2:
        raise ValueError("Input must be a tuple with two elements: (titles, descriptions).")

    titles, descriptions = news

    if not (isinstance(titles, list) and isinstance(descriptions, list)):
        raise ValueError("Both titles and descriptions must be lists.")

    if len(titles) != len(descriptions):
        raise ValueError("Titles and descriptions lists must have the same length.")

    sentiment_record = {"positive": 0, "negative": 0, "neutral": 0}

    # Perform sentiment analysis
    for title, description in zip(titles, descriptions):
        text = f"{title} {description}"
        analysis = TextBlob(text).sentiment.polarity

        if analysis > 0:
            sentiment_record["positive"] += 1
        elif analysis < 0:
            sentiment_record["negative"] += 1
        else:
            sentiment_record["neutral"] += 1

    print(sentiment_record)

    # Determine final sentiment
    if sentiment_record["positive"] > max(sentiment_record["negative"], sentiment_record["neutral"]):
        return "POSITIVE"
    elif sentiment_record["negative"] > max(sentiment_record["positive"], sentiment_record["neutral"]):
        return "NEGATIVE"
    else:
        return "NEUTRAL"




In [47]:
symbol_list = (nse_df['symbol']).tolist()

In [ ]:
NEWS = []

for symbol in symbol_list:
    try:
        news = scrape_yahoo_finance_news(symbol)
        result = news_analysis(news)
        NEWS.append(result)
        time.sleep(1)  # Add delay between requests
    except Exception as e:
        print(f"Error processing {symbol}: {str(e)}")
        NEWS.append("NEUTRAL")  # Default fallback

#### Continue Here

In [ ]:
nse_df["News"] = NEWS
nse_df.head()

In [ ]:
nse_df.head()

In [ ]:
# Downloading the above data as excel sheet

file_path = " Today's_data.xlsx"
date = datetime.now().date()
date = date.strftime("%d-%m-%Y")
file_path = date + file_path
nse_df.to_excel(file_path, index = False)
print("File downloaded")